# Armani SageMaker instance 

In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
# from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the eu-north-1 region. You will use the 669576153137.dkr.ecr.eu-north-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [2]:
bucket_name = 'cestino-s3'
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


The part until now is equal to the [tutorial](https://aws.amazon.com/it/getting-started/hands-on/build-train-deploy-machine-learning-model-sagemaker/).

In [3]:
try:
  urllib.request.urlretrieve("https://leonardo-michelazzo.github.io/LLama-AI/categorizzatore/Armani.csv", "./products.csv")
  urllib.request.urlretrieve("https://leonardo-michelazzo.github.io/LLama-AI/categorizzatore/Armani-train.csv", "./train.csv")
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./products.csv', header=None, index_col=False) # Changed this
  train_data = pd.read_csv('./train.csv', header=None, index_col=False) # Added this
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.


Now that we have loaded into model_data the CSV...

In [4]:
print(model_data) # ...and after a bit of debugging...
print(type(model_data)) # ...and compensation of the absurd leak of documentation...

    0  1   2   3  4   5   6  7
0   0  0   0   0  0   0   0  0
1   0  0   0   0  0   3  43  3
2   0  1   1   0  0   3  44  1
3   0  1   2   1  0   3  42  3
4   0  7   3   2  0   0  38  1
5   0  3   4   3  1   4  43  1
6   0  0   2   1  0   3  43  5
7   0  6  14   2  0   0  38  1
8   0  6   4   4  1   4  38  1
9   0  6   4   5  2   7  38  1
10  0  6   4  13  0   0  38  1
11  0  0   8   6  1   1  43  5
12  0  1   8   7  1   8  46  1
13  0  7   4   2  1   0  39  1
14  0  3   5  12  1   2  41  6
15  0  7   9   2  1   0  39  1
16  0  7   9   2  1   6  38  1
17  0  3   0   8  1   5  43  1
18  0  6   6   8  1   5  39  1
19  0  5   0   9  0   0  43  0
20  0  1  10   9  0   0  43  1
21  0  0   0   0  0   0  45  3
22  1  0   3   9  0   0  51  3
23  1  1   3   9  0   0  51  1
24  1  0   0   9  0   0  50  3
25  1  1   0   9  0   0  48  1
26  1  0   8   8  0   0  52  4
27  1  1   8   8  0   0  52  2
28  1  2   0   9  0   0  41  1
29  1  2   0   1  0   0  41  2
30  1  2   2   1  0   0  40  1
31  1  1

...we start splitting data into the data for training (in this case 70% of the total) an data for testing (the rest).

In [5]:
# train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
# train_data, test_data = model_data, model_data
test_data = model_data
print(train_data.shape, test_data.shape)

(51, 9) (51, 8)


We should first get what [random_state](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) is.
Well, it's just a randomizer I guess.

In [6]:
print(train_data)
print(test_data)

    0  1   2   3  4   5   6  7  8
0   0  0   0   0  0   0   0  0  0
1   0  0   0   0  0   3  43  3  0
2   0  1   1   0  0   3  44  1  0
3   0  1   2   1  0   3  42  3  1
4   0  7   3   2  0   0  38  1  0
5   0  3   4   3  1   4  43  1  1
6   0  0   2   1  0   3  43  5  0
7   0  6  14   2  0   0  38  1  0
8   0  6   4   4  1   4  38  1  0
9   0  6   4   5  2   7  38  1  0
10  0  6   4  13  0   0  38  1  1
11  0  0   8   6  1   1  43  5  0
12  0  1   8   7  1   8  46  1  0
13  0  7   4   2  1   0  39  1  1
14  0  3   5  12  1   2  41  6  1
15  0  7   9   2  1   0  39  1  0
16  0  7   9   2  1   6  38  1  0
17  0  3   0   8  1   5  43  1  0
18  0  6   6   8  1   5  39  1  0
19  0  5   0   9  0   0  43  0  0
20  0  1  10   9  0   0  43  1  0
21  0  0   0   0  0   0  45  3  0
22  1  0   3   9  0   0  51  3  0
23  1  1   3   9  0   0  51  1  1
24  1  0   0   9  0   0  50  3  0
25  1  1   0   9  0   0  48  1  1
26  1  0   8   8  0   0  52  4  0
27  1  1   8   8  0   0  52  2  0
28  1  2   0  

In [7]:
train_data.to_csv(
  'train.csv',
  index=False,
  header=False
)
boto3.Session().resource('s3').Bucket(bucket_name).Object(
    os.path.join(
        prefix, 'train/train.csv'
    )
).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(
    s3_data=f's3://{bucket_name}/{prefix}/train',
    content_type='csv'
)

In [8]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(
    xgboost_container,role, instance_count=1,
    instance_type='ml.m5.4xlarge',
    output_path=f's3://{bucket_name}/{prefix}/output',
    sagemaker_session=sess
)
xgb.set_hyperparameters(
    max_depth=5,eta=0.2,gamma=4,
    min_child_weight=6,subsample=0.8,
    silent=0,objective='binary:logistic',
    num_round=100
)

In [9]:
print(s3_input_train)

In [10]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2023-06-26-13-45-00-021


2023-06-26 13:45:00 Starting - Starting the training job...
2023-06-26 13:45:21 Starting - Preparing the instances for training......
2023-06-26 13:46:21 Downloading - Downloading input data...
2023-06-26 13:46:51 Training - Downloading the training image...
2023-06-26 13:47:27 Uploading - Uploading generated training modelArguments: train
[2023-06-26:13:47:20:INFO] Running standalone xgboost training.
[2023-06-26:13:47:20:INFO] Path /opt/ml/input/data/validation does not exist!
[2023-06-26:13:47:20:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 55603.2mb
[2023-06-26:13:47:20:INFO] Determined delimiter of CSV input is ','
[13:47:20] S3DistributionType set as FullyReplicated
[13:47:20] 51x8 matrix with 408 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[13:47:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[0]#011train-error:0.431373
[13:47:20] src/tre

Then we'll instanciate the sagemaker.predictor.Predictor...

In [ ]:
predictor = xgb.deploy(
    initial_instance_count=1,
    instance_type='ml.r5.large'
)

INFO:sagemaker:Creating model with name: xgboost-2023-06-26-13-55-56-809
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-06-26-13-55-56-809
INFO:sagemaker:Creating endpoint with name xgboost-2023-06-26-13-55-56-809


---

In [12]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.values #load the data into an array
predictor.serializer = CSVSerializer() # set the serializer type
predictions = predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "Unable to evaluate payload provided: Feature size of csv inference data 8 is not consistent with feature size of trained model 7". See https://eu-north-1.console.aws.amazon.com/cloudwatch/home?region=eu-north-1#logEventViewer:group=/aws/sagemaker/Endpoints/xgboost-2023-06-26-12-58-24-473 in account 191548338305 for more information.